<a href="https://colab.research.google.com/github/tushar-navale/GenAI_project/blob/main/GenAI_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:


!pip install datasets rank-bm25 torch torchaudio transformers diffusers google-colab


In [10]:
from datasets import load_dataset
from rank_bm25 import BM25Okapi
import torch
import torchaudio
import torchaudio.transforms as transforms
import numpy as np
from transformers import T5Tokenizer, T5ForConditionalGeneration
from diffusers import UNet2DConditionModel, DDPMScheduler
from google.colab import files

In [12]:
def bm25_retrieve(query, bm25, stories, k=5):
    query_tokens = query.lower().split()
    scores = bm25.get_scores(query_tokens)
    best_indices = np.argsort(scores)[-k:][::-1]
    return [stories[i] for i in best_indices]



In [13]:
def generate_music(text, tokenizer, model):
    input_ids = tokenizer(text, return_tensors='pt').input_ids
    output = model.generate(input_ids)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [14]:
def refine_music_with_diffusion(music_embedding, unet, scheduler):
    noise = torch.randn_like(music_embedding)
    scheduler.set_timesteps(50)
    for t in scheduler.timesteps:
        noise = unet(noise, t)['sample']

In [21]:
def load_and_preprocess_data():
    with open("/content/train.txt", "r") as file:
        stories = file.read().strip().split('\n')
    tokenized_stories = [story.lower().split() for story in stories]
    return stories, tokenized_stories


stories, tokenized_stories = load_and_preprocess_data()

# BM25 Retrieval
bm25 = BM25Okapi(tokenized_stories)
query = "A thrilling adventure in the mountains"
retrieved_texts = bm25_retrieve(query, bm25, stories)

# Load Transformer Model for music generation
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Generate music embeddings
music_embeddings = [generate_music(text, tokenizer, model) for text in retrieved_texts]

# Convert to tensor representation
music_tensor = torch.tensor([hash(embed) % 1000 for embed in music_embeddings], dtype=torch.float32)

# Load Diffusion Model for refinement
unet = UNet2DConditionModel.from_pretrained("CompVis/ldm-text2im-large-256")
scheduler = DDPMScheduler(num_train_timesteps=1000)

# Refine music with diffusion
refined_music = refine_music_with_diffusion(music_tensor, unet, scheduler)

# Convert refined tensor to waveform
waveform = refined_music.numpy()
wav_output = torch.from_numpy(waveform).unsqueeze(0)

# Save as audio
torchaudio.save("generated_music.wav", wav_output, 16000)

# Download the file in Google Colab
files.download("generated_music.wav")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

OSError: CompVis/ldm-text2im-large-256 does not appear to have a file named config.json.